In [2]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import pandas as pd
import pathlib
import shutil
import sqlalchemy as sa

import database as db

from images import copy_images, get_list_of_images_invalid, remove_images_invalid
from tables import County, DataTrustedIdentifier, InfoImage

## Database connection

In [5]:
engine, session = db.connect()
engine.echo = False

2023-03-23 16:16:15,802 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-23 16:16:15,803 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-23 16:16:15,804 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-23 16:16:15,805 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-23 16:16:15,806 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-23 16:16:15,807 INFO sqlalchemy.engine.Engine [raw sql] {}


In [24]:
list_images = [str(image) for image in pathlib.Path('/home/xandao/Documentos/RGB/512/w_pred_mask').rglob('*.jpeg') if image.is_file()]
print('count of images is a %d' % len(list_images))

count of images is a 52604


In [25]:
query = session.query(County).filter(County.uf=='PR').distinct().all()

uf_unaccented_lower, state_unaccented_lower, county_unaccented_lower = db.get_list_uf_state_county(query)

In [40]:
level = DataTrustedIdentifier.specific_epithet
columns = [level, sa.func.array_agg(DataTrustedIdentifier.seq)]
condition = sa.and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                        DataTrustedIdentifier.specific_epithet.is_not(None),
                        DataTrustedIdentifier.specific_epithet.is_not(None),
                        sa.or_(uf_unaccented_lower, state_unaccented_lower))
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens'

In [41]:
list_images_invalid = get_list_of_images_invalid()
pd.DataFrame(list_images_invalid)

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal
3,NY01421575_01,horizontal
4,HUFSJ001689_v00,not exsicate
5,HUFSJ001133_v00,not exsicate
6,HUFSJ002198_v00,not exsicate
7,HUFSJ003255_v00,not exsicate
8,HVASF000487_v01,not exsicate
9,INPA0019084_nd,label


In [43]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            records = db.get_records_group_by_level(columns, condition, level, minimum_image, session)
            list_level_name, list_path_images = db.filter_records(color, image_size, minimum_image, records, session)
            list_count_path, list_path_images = remove_images_invalid(list_level_name, list_images_invalid, list_path_images)
            copy_images(color, image_size, list_level_name, list_path_images, str(minimum_image), path_out)

            df = pd.DataFrame({
                'species': list_level_name,
                'paths': list_path_images,
                'count': list_count_path
            })

            display('color: %s image_size: %s minimum_image: %d' % (color, image_size, minimum_image))
            display('len(list_level_name): %d len(paths_images): %d' % (len(list_level_name), df['count'].sum()))
            display(df.head(3))

/home/xandao/Imagens/RGB/256/5


'color: rgb image_size: 256 minimum_image: 5'

'len(list_level_name): 0 len(paths_images): 0'

,species,paths,count


/home/xandao/Imagens/RGB/256/10


...

,species,paths,count


/home/xandao/Imagens/GRAYSCALE/512/20


'color: grayscale image_size: 512 minimum_image: 20'

'len(list_level_name): 0 len(paths_images): 0'

,species,paths,count


## Close connection

In [63]:
session.close()
engine.dispose()